### First Try and Initialization

In [2]:
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool

In [4]:
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    api_key="1122",
    num_ctx=8192,
)

In [5]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model, add_base_tools=True)


In [ ]:
agent.run("What is the capital of India?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of India?                                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5:7b ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = web_search(query="capital of India")                                                                    
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
New Delhi (/ ˈ nj uː ˈ d ɛ. l i / ⓘ; [6] Hindi: Naī Dillī , pronounced [ˈnəiː ˈdɪlːiː]) is the capital of India and
a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the 
Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court.New Delhi is a 
municipality within the NCT, administered by the New ...

[List of state and union territory capitals in India - 
Wikipedia](https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India)
India is a federal constitutional republic governed under a parliamentary system consisting of 28 states and 8 
union territories. [1] All states, as well as the union territories of Jammu and Kashmir, Puducherry and the 
National Capital Territory of Delhi, have elected legislatures and governments, both patterned on the Westminster 
model.The remaining five union territories are directly ruled ...

[New Delhi | History, Population, Map, & Facts | Britannica](https://www.britannica.com/place/New-Delhi)
New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of 
the Yamuna River, adjacent to and just south of Delhi city (Old Delhi) and within the Delhi national capital 
territory. Learn more about New Delhi in this article.

[States and Capitals of India, 28 States and 8 UTs in India (Updated in 
...](https://www.careerpower.in/states-and-capitals-of-india.html)
States and Capitals of India. India is the 7th largest country in the world and also the 2nd most populous. India 
is a diverse and culturally rich Nation that comprises 28 states and 8 Union territories, each with its own unique 
identity, history, and administrative center. India lies in the southern Asia.

[Delhi | History, Population, Map, & Facts | Britannica](https://www.britannica.com/place/Delhi)
Delhi is a city and national capital territory in north-central India. The city of Delhi actually consists of two 
components: Old Delhi, in the north, the historic city; and New Delhi, in the south, since 1947 the capital of 
India, built in the first part of the 20th century as the capital of British India.

[List of capitals of India - Wikipedia](https://en.wikipedia.org/wiki/List_of_capitals_of_India)
In 1858, Allahabad (now Prayagraj) became the capital of India for a day when it also served as the capital of 
North-Western Provinces. [3]During the British Raj, until 1911, Calcutta was the capital of India. [4]By the latter
half of the 19th century, Shimla had become the summer capital. [5]King George V proclaimed the transfer of the 
capital from Calcutta to Delhi at the climax of the 1911 ...

[What is the Capital of India? - WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-india.html)
Learn about the history, geography, and problems of New Delhi, the capital city of India. Find out why and how the 
capital was moved from Kolkata to New Delhi in 1911.

[What is the Capital of India? - Mappr](https://www.mappr.co/capital-cities/india/)
Learn about the history, geography, culture, and attractions of New Delhi, the capital of India. Find out how New 
Delhi became the capital in 1912, what is the difference between Delhi and New Delhi, and what is the climate and 
population of the city.

[What is the capital of India? States and union territories 
explained.](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)
New Delhi is part of one of India's union territories, the National Capital Territory of Dehli. Old Delhi , now a 
historic city, is north of New Delhi in the same union territory.

[What is the capital city of India? - 
GeeksforGeeks](https://www.geeksforgeeks.org/what-is-the-capital-city-of-india/)
The capital city of India is New Delhi, situated in the northern part of the country. It was officially declared 
t

[Step 1: Duration 138.41 seconds| Input tokens: 2,141 | Output tokens: 40]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━